# Introduction
In order to make the Recommender System created usable it has been decided to provide a demo of the real use of it.

Specifically it has been decided to create a web application thanks to the Python library *Streamlit*.

The choice behind the use of Streamlit is the fact that is a very straightforward library to create web application that make use of ML models.

In the following sections of this notebook the passage needed to run the web application are presented: specifically, all the needed libraries are imported (and if needed installed), the local tunnel needed to go to the website is created and the web application is written in the *app.py* file.

In the end the commands needed to acces to the website are presented.

# Needed Library Import

In [ ]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 27.2 MB/s eta 0:00:00


In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.572s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=94584d506a8cc5096721fbff832e3a572911207c026cf1390b2536e2a918faaa
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


# Connection to Google Drive

In [ ]:
# Connect this colab to Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Demo

In [ ]:
%%writefile app.py
# Import libraries to execute the application functionalities
import streamlit as st
import pyspark
import pandas as pd
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql import functions as sqlf
from pyspark import SparkContext, SparkConf
from pyspark.ml.recommendation import ALSModel

# Create the Spark Session
conf = SparkConf().setAppName("BDC_Demo").setMaster("local")
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# Application Title
st.title('Videogame Recommender System')

# Load all needed models and data
als_model = ALSModel.load('/content/gdrive/MyDrive/VideogameRecommenderSystem/models/ALS19_08_2023__13_09_01')
demo_df = spark.read.load('/content/gdrive/MyDrive/VideogameRecommenderSystem/demo_data',
                           format="csv",
                           sep=",",
                           inferSchema="true",
                           header="true")
demo_cluster = spark.read.load('/content/gdrive/MyDrive/VideogameRecommenderSystem/demo_cluster',
                           format="csv",
                           sep=",",
                           inferSchema="true",
                           header="true")
demo_game_cluster = spark.read.load('/content/gdrive/MyDrive/VideogameRecommenderSystem/demo_game_cluster_data',
                           format="csv",
                           sep=",",
                           inferSchema="true",
                           header="true")
games_df = spark.read.load('/content/gdrive/MyDrive/VideogameRecommenderSystem/games.csv',
                           format="csv",
                           sep=",",
                           inferSchema="true",
                           header="true")

def compute_ALS_recommendation(steam_id):
  '''
  This function is used to compute the recommendation for the user with ID
  steam_id with the ALS model
  '''
  st.write('Computing the recommendations based on the item you already liked...')
  steam_int_id = demo_df.filter(demo_df.steam_id == steam_id).first()['steam_int_id']
  user = spark.createDataFrame([steam_int_id], "int").toDF("steam_int_id")
  user_recs = als_model.recommendForUserSubset(user, 5)
  # The function to display the results is called
  display_ALS_recommendation(user_recs)
  return

def display_ALS_recommendation(recommendation_df, games_info=games_df):
  '''
  This function is used to display the recommendation previously computed with
  the ALS model.
  The results are displayed in a table
  '''
  st.write('Displaying the information about the recommendations computed...')
  data = recommendation_df.first()
  recommendations_value = data["recommendations"]
  recommendations = [{row.steam_appid : row.rating} for row in recommendations_value]
  recommendations_info = []
  for recommendation in recommendations:
    data = []
    appid = list(recommendation.keys())[0]
    title = games_info.filter(games_info.steam_appid == appid).first()['name']
    data.append(title)
    value = recommendation[appid]
    data.append(value)
    description = games_info.filter(games_info.steam_appid == appid).first()['short_description']
    data.append(description)
    # All the links to pages of application in steam follow the same structure
    link = 'https://store.steampowered.com/app/{}'.format(appid)
    data.append(link)
    recommendations_info.append(data)
  columns = ['Title', 'Rating', 'Description', 'Steam Page']
  df = pd.DataFrame(recommendations_info, columns=columns)
  st.table(df)
  return

def compute_cluster_recommendation(title):
  '''
  This function is used to compute the recommendation for the videogame with title
  title with the Clustering model
  '''
  st.write('Finding similar games to the one you liked...')
  cluster = demo_cluster.filter(demo_cluster.name == title).first()['cluster']
  games_in_cluster = demo_cluster.filter(sqlf.col('cluster') == cluster)
  # 5 random recommendation of games present in the same cluster are provided to
  # the user
  recommendations = games_in_cluster.sample(False, fraction=5/games_in_cluster.count())
  # The function to display the result is called
  display_cluster_recommendation(recommendations)
  return

def display_cluster_recommendation(recommendation_df, games_info=games_df):
  '''
  This function is used to display the recommendation previously computed with
  the clustering model.
  The results are displayed in a table
  '''
  st.write('Displaying the information about the sinilar games...')
  recommendations_info = []
  recommendations = recommendation_df.collect()
  for recommendation in recommendations:
    data = []
    title = recommendation['name']
    data.append(title)
    appid = recommendation['steam_appid']
    description = games_info.filter(games_info.steam_appid == appid).first()['short_description']
    data.append(description)
    link = 'https://store.steampowered.com/app/{}'.format(appid)
    data.append(link)
    recommendations_info.append(data)
  columns = ['Title', 'Description', 'Steam Page']
  df = pd.DataFrame(recommendations_info, columns=columns)
  st.table(df)
  return


st.write("What do you care about the most?\n Others' opinions or the story that you are going to live?")
choice = st.selectbox('Select your choice', options=["", "Others' opinions", "The story to live"])

if choice == "Others' opinions":
  id_list = [row.steam_id for row in demo_df.select('steam_id').distinct().collect()]
  steam_id = st.selectbox('Select the Steam ID', options=['', *id_list])
  btn_als = st.button("Compute Recommendations!")
  if btn_als:
    compute_ALS_recommendation(steam_id)
elif choice == "The story to live":
  game_list = [row.name for row in demo_game_cluster.select('name').distinct().collect()]
  game = st.selectbox('Select one game you liked', options=['', *game_list])
  btn_cluster = st.button("Compute Recommendations!")
  if btn_cluster:
    compute_cluster_recommendation(game)
else:
  st.write("Please select one of the possible options")



Overwriting app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!curl ipv4.icanhazip.com

35.196.216.142


In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 1.676s
your url is: https://ready-wings-go.loca.lt
^C
